Author: Dominique Grimes

Date: July 21, 2024

Topic: Sales prediction

In [ ]:
# Load libraries
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import pearsonr
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import association_metrics as am
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
from pandas.errors import SettingWithCopyWarning
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import SimpleExpSmoothing
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance, plot_tree
# plt.style.use('fivethirtyeight')

In [ ]:
# Set options to see max columns and rows
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Supress warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load data into df
sales = pd.read_csv('Amazon Sale Report.csv')

In [ ]:
# Verify data loaded 
sales.head()

# Data Cleansing

In [ ]:
sales.info()

In [ ]:
# Rename column headers that have spaces
sales = sales.rename(columns={"Order ID": "Order_ID", "Sales Channel ": "Sales_Channel", "Courier Status": "Courier_Status", "Unnamed: 22": "Unnamed"})

In [ ]:
# Verify new column headers
sales.head()

## Observe and manage null values

In [ ]:
# Counting NaN values in all columns
nan_count = sales.isna().sum()

In [ ]:
nan_count

Initial thoughts
- Courier_Status may be correlated with Status. If so, drop.
- Amount needs to be observed for why the values are null. May be dependent on another feature. If not, replace price with average price for similar style clothing.
- Ship postal code may be due to cancellations. If not, impute the mode.
- I simplified this column into a binary feature to observe if a promotion was or was not applied. 
- fulfilled-by can be dropped since this item is dependent on Fulfilment.

## Observe value counts for each categorical feature

In [ ]:
sales['Status'].value_counts()

In [ ]:
sales['Date'].value_counts()

In [ ]:
sales['Fulfilment'].value_counts()

In [ ]:
sales['Sales_Channel'].value_counts()

In [ ]:
sales['ship-service-level'].value_counts()

In [ ]:
sales['Courier_Status'].value_counts()

In [ ]:
sales['Style'].value_counts()

In [ ]:
sales['Category'].value_counts()

In [ ]:
sales['Size'].value_counts().sort_index()

In [ ]:
sales['Qty'].value_counts().sort_index()

In [ ]:
sales['currency'].value_counts()

In [ ]:
sales['ship-city'].value_counts()

In [ ]:
sales['ship-state'].value_counts()

In [ ]:
sales['ship-state'] = sales['ship-state'].str.upper()

In [ ]:
sales['ship-state'].value_counts().sort_index()

In [ ]:
sales['ship-country'].value_counts()

In [ ]:
sales['promotion-ids'].value_counts()

In [ ]:
sales['fulfilled-by'].value_counts()

In [ ]:
sales['Unnamed'].value_counts()

In [ ]:
sales['B2B'].value_counts()

## Drop columns with little to no variation

In [ ]:
# Check if Order_ID is unique. 
sales['Order_ID'].is_unique

In [ ]:
sales['Order_ID'].nunique()

In [ ]:
# Drop unnecessary columns
sales.drop(['Order_ID','index','ship-country', 'Unnamed','currency','ship-city','ship-state','SKU'], axis=1, inplace=True)

In [ ]:
# Verify df shape
sales.shape

### promotion-ids

In [ ]:
# Change promotion-ids to binary
sales['promotion-ids'] = sales['promotion-ids'].where(~sales['promotion-ids'].notna(), 1)
sales['promotion-ids'] = sales['promotion-ids'].fillna(0)

In [ ]:
# Rename column headee for promotion-ids to promotion
sales = sales.rename(columns={"promotion-ids": "promotion"})

In [ ]:
# Verify column header change
sales.head(1)

### ship-postal-code

In [ ]:
# View number of unique postal codes
sales['ship-postal-code'].nunique()

In [ ]:
# selecting rows where postal code is na 
null_pc = sales[sales['ship-postal-code'].isna()]

In [ ]:
# View null postal code df
null_pc

I did not observe any obvious relationships between the null postal codes and the rest of the features. I will replace the 33 null postal codes with the mode zip code.

In [ ]:
# Find the mode of the postal code feature
sales['ship-postal-code'].mode()

In [ ]:
# Impute na values for ship-postal-code with mode 201301.0
sales['ship-postal-code']=sales['ship-postal-code'].fillna(201301.0)

In [ ]:
# Counting NaN values in all columns
nan_count = sales.isna().sum()
nan_count

### Courier_Status

Observe when there are zero quantities purchased how much the sales are for 0 quantity.
Look at relationship between qty and courrier status.
Assume if NAN then 0 qty.This is due to being cancelled.
Look at relationship between status and courier status
courier status: Observe shipped, unshipped, cancelled behavior. 

In [ ]:
# Review value counts for Courier_Satus again. Note: 6,872 null values
sales['Courier_Status'].value_counts()

In [ ]:
# Review value counts for Qty again focusing sales with zero Qty
sales['Qty'].value_counts()

In [ ]:
# Filter the sales df with values Qty = 0
filtered_sales = sales[sales['Qty'] == 0] 

In [ ]:
# View value counts of the filtered data frame for all courier statuses when Qty = 0
filtered_sales['Courier_Status'].value_counts()

In [ ]:
# Count NaN values of filtered df and observe number of null values for Courier_Status
nan_count1 = filtered_sales.isna().sum()
nan_count1

Total values of filtered dataframe when Qty = 0 is 6872 null Carrier Status + 5935 Cancelled Status = 12807 of total Qty 0 in the unfiltered dataframe. I'm chosing to replace null Courier_Status with Cancelled. 

In [ ]:
# Impute na values for Courier_Status with Cancelled
sales['Courier_Status']=sales['Courier_Status'].fillna('Cancelled')

In [ ]:
# Verify replaced na values
sales['Courier_Status'].value_counts()

In [ ]:
# Counting NaN values in all columns
nan_count2 = sales.isna().sum()
nan_count2

### Fulfilled-by

In [ ]:
fulfilledna = sales[sales['fulfilled-by'].isna()]

In [ ]:
fulfilledna['Fulfilment'].value_counts()

In [ ]:
fulfilledna = sales[sales['fulfilled-by'] == 'Easy Ship']

In [ ]:
fulfilledna['Fulfilment'].value_counts()

Fulfilment is binary with Amazon or Merchant. This is directly related to fulfilled-by. If not fulfilled by Amazon, then the merchant is Easy Street. Since this is the case, I will drop fulfilled-by and change Fulfilment to a binary feature in the Feature Engineering section.

### Amount

In [ ]:
# selecting rows where Amount is na 
null_Amount = sales[sales['Amount'].isna()]

In [ ]:
# View filtered data
null_Amount

In [ ]:
# Check the row count of the filtered data
null_Amount.shape

In [ ]:
# Check the number of each category for Courier_Status
null_Amount['Courier_Status'].value_counts()

There are no Shipped values when Amount is null. Most are cancelled with a vew unshipped. Observe if the qty is 0 for these and what the price Amount is for cancelled items.

In [ ]:
# Counts of each quantity when Amounts are null
null_Amount['Qty'].value_counts()

The majority of the null Amounts have a qty of zero. 

In [ ]:
# Create a df with Amount and Category
Cat_Amt = sales[['Category', 'Amount']]

In [ ]:
# View new df
Cat_Amt.head()

In [ ]:
average_Amount = sales.groupby('Category')['Amount'].mean().round(2)

In [ ]:
# View the average sales per category
Cat_ave_Amount = average_Amount.round(2)
Cat_ave_Amount

In [ ]:
# Replace null values with the average value of each category
sales['Amount'] = sales.groupby('Category')['Amount'].transform(lambda x: x.fillna(x.mean()))
sales.head()

In [ ]:
# Check the null valies for Amount
sales['Amount'].isna().sum()

None of the Amounts are zero when the Courier_Status is cancelled. I will investigate why there are zero amounts that are not cancelled in the Analysis of Continuous variables section. 

## Feature Engineering

### Create binary features

In [ ]:
# Calling the get_dummies method. The first parameter mentions the the name of the data frame to store the 
# new data frame in the second parameter is the list of columns which if not mentioned returns the dummies for 
# binomial columns 

sales_dummies = pd.get_dummies(sales, columns = ['Fulfilment', 'B2B', 'ship-service-level','Sales_Channel']) 

In [ ]:
# Drop 2nd dummy column of binomial features
sales_dummies.drop(['Fulfilment_Merchant','B2B_False', 'ship-service-level_Standard','Sales_Channel_Non-Amazon', 'fulfilled-by'], axis=1, inplace=True)

In [ ]:
# View df to verify the feature was dropped
sales_dummies.head(3)

#### Split Status Column and create Returned binary feature

In [ ]:
# View value counts for the items in the Status column
sales['Status'].value_counts()

In [ ]:
# Create two new features that split the column values on the "-"
sales_dummies[['Status2', 'Status3']] = sales_dummies['Status'].str.split(' - ', n=1, expand=True)

In [ ]:
# Rename Status3 to Returned
sales_dummies.rename(columns={"Status3": "Returned"}, inplace=True)

In [ ]:
# Transform Status 3 into binary feature and update values to 0 or 1 accordingly
sales_dummies['Returned'].replace({'Delivered to Buyer':0, 'Returned to Seller':1, 'Picked Up':0,'Waiting for Pick Up':0, 'Returning to Seller':1, 'Out for Delivery':0, 'Rejected by Buyer':1, 'Lost in Transit':0, 'Damaged':1, 'None':0}, inplace=True)

In [ ]:
# View value counts for new Returned feature
sales_dummies['Returned'].value_counts()

In [ ]:
# Fill null Returned values with 0
sales_dummies['Returned'].fillna(0, inplace = True)

In [ ]:
# Observe value counts after null vaues replaced
sales_dummies['Returned'].value_counts()

In [ ]:
# Verify there are no remaining null returned values
sales_dummies['Returned'].isna().sum()

In [ ]:
# Update the dtype of Returned to int
sales_dummies['Returned'] = sales_dummies['Returned'].astype(int)

In [ ]:
# Verify the dtype was updated
sales_dummies['Returned'].dtype

### Date Column

In [ ]:
# Convert Date to datetime
sales_dummies['Date'] = pd.to_datetime(sales_dummies['Date'],format='%m-%d-%y')

In [ ]:
# Split the feature into mont, day, year, dayofweek, and week features
sales_dummies['month'] = sales_dummies['Date'].dt.month
sales_dummies['day'] = sales_dummies['Date'].dt.day
sales_dummies['year'] = sales_dummies['Date'].dt.year
sales_dummies['dayofweek'] = sales_dummies['Date'].dt.dayofweek
sales_dummies['week'] = sales_dummies['Date'].dt.week

In [ ]:
# Verify the new features were created
sales_dummies.head(1)

In [ ]:
# Review the values for year
sales_dummies['year'].value_counts()

I will drop year since no variation as well as Date since the infomation has been spit into feature columns

In [ ]:
# Drop date and year features
sales_dummies.drop('year', axis=1, inplace=True)

In [ ]:
sales_dummies['Category'] = sales_dummies['Category'].astype('category')
sales_dummies['Size'] = sales_dummies['Size'].astype('category')
sales_dummies['Qty'] = sales_dummies['Qty'].astype('int')
sales_dummies['ship-postal-code'] = sales_dummies['ship-postal-code'].astype('int')

In [ ]:
sales_dummies['Courier_Status'].value_counts()

In [ ]:
sales_dummies.dtypes

In [ ]:
sales_dummies['Category'].value_counts().sort_index()

# Exploratory Data Analysis

## Continuous feature Amount

In [ ]:
sales['Amount'].describe()

In [ ]:
# Observe distribution of Amount through histogram

# Plotting a basic histogram
plt.hist(sales['Amount'], bins=100, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Sales Values')
plt.ylabel('Frequency')
plt.title('Histogram of Amazon Sales Amounts (INR)')
 
# Display the plot
plt.show()

In [ ]:
# Observe distribution of Amount through boxplot
fig = px.box(sales, y="Amount", title='Boxplot of Amount')
fig.show()

Even though there are some outliers, they seem like relevant values that will impact sales. I am chosing not to remove them at this time.

Explore if cancelled items have zero amounts

In [ ]:
# Filter data frame for Courier_Status of cancelled
can = sales[sales['Courier_Status']=='Cancelled']

In [ ]:
# Oberve the Amounts for cancelled items that are zero. 
len(can[can['Amount'] == 0])

There are no cancelled items when the amount is zero.

In [ ]:
# Observe the lenth of items in the original df where Amount is zero
len(sales[sales['Amount'] == 0])

In [ ]:
# Create df that filters the data with zero amounts
zeroAMT = sales[sales['Amount'] == 0]

In [ ]:
# View df
zeroAMT.head()

There is nothing obvious causing the to zero amounts like cancelled items or zero qty. This seems to be an error. I will replace zero and null Amounts with the average prices per category.

In [ ]:
# Replace zero amounts with the mean
sales['Amount'] = sales.groupby('Category')['Amount'].transform(lambda x: x.replace(0,x.mean()))

In [ ]:
# Check the count of zero Amounts after imputation
len(sales[sales['Amount'] == 0])

In [ ]:
# Verify the mean Amounts per category after imputation
check = sales.groupby('Category')['Amount'].mean().round(2)
check

In [ ]:
# Compare to the original mean Amounts per category 
Cat_ave_Amount

In [ ]:
# Observe distribution of Amount through histogram with encoded data

# Plotting a basic histogram
plt.hist(sales['Amount'], bins=100, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Sales Values')
plt.ylabel('Frequency')
plt.title('Histogram of Amounts after Zero Imputation')
 
# Display the plot
plt.show()

The new mean amounts are slightly higher which is to be expected since the amounts are skewed by the inaccurate 0 Amounts. 

## Time related distribution analysis

In [ ]:
# Create dataframe that sorted by date in ascending order
sorted_time = sales_dummies.sort_values(by = ['Date'], ascending = True)
sorted_time.head(10)

In [ ]:
# Create a df with total sales per day
date_df = sorted_time.groupby(sorted_time['Date'])['Amount'].sum().reset_index()

In [ ]:
# View the df
date_df.tail()

In [ ]:
# Plot the time series sales data
plt.figure(figsize=(15,5))
plt.plot(date_df['Date'],date_df['Amount'], color='skyblue')

# Name the x axis 
plt.xlabel('Date') 
# Name the y axis 
plt.ylabel('Total Retail Sales per day') 
  
# Add graph title
plt.title('Total Retail Sales per Day from March 31, 2022 to June 29, 2022') 
plt.xticks(rotation=45)

# show the plot 
plt.show() 

In [ ]:
# View the value counts of the month feature
sales_dummies['month'].value_counts()

In [ ]:
# Review the distribution of day feature through value counts
# Create a df that provides the value counts for each day
day=sales_dummies['day'].value_counts().sort_index()

In [ ]:
#Observe distribution of Date through histogram

# Plotting a basic histogram
plt.hist(sales_dummies['day'], bins=30, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Date of Sales')
plt.ylabel('Frequency')
plt.title('Histogram of Amazon Sales Date')
 
# Display the plot
plt.show()

In [ ]:
# Observe distribution of day of week counts through value counts
sales_dummies['dayofweek'].value_counts().sort_index()

In [ ]:
# Observe distribution of day of week through histogram

# Plotting a basic histogram
plt.hist(sales_dummies['dayofweek'], bins=7, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Date of Sales')
plt.ylabel('Frequency')
plt.title('Histogram of Amazon Sales Day of Week')
 
# Display the plot
plt.show()

In [ ]:
# Observe distribution of week of the year counts through value counts
sales_dummies['week'].value_counts().sort_index()

In [ ]:
#Observe distribution of week of the year through histogram

# Plotting a basic histogram
plt.hist(sales_dummies['week'], bins=14, color='skyblue', edgecolor='black')
 
# Adding labels and title
plt.xlabel('Date of Sales')
plt.ylabel('Frequency')
plt.title('Histogram of Amazon Sales Week of Year')
 
# Display the plot
plt.show()

Week of the year distribution looks bimodal

## Correlations

#### Observe if there are correlations between Status2 and Courier_Status.

In [ ]:
# Compare status2 and Courier_Status

# Status2 value counts
sales_dummies['Status2'].value_counts(dropna=False)

In [ ]:
# Courier_Status value counts
sales_dummies['Courier_Status'].value_counts(dropna=False)

In [ ]:
# Convert str columns to Category columns
df = sales_dummies[{'Status2','Courier_Status'}]
df = df.apply(lambda x: x.astype("category") if x.dtype == "O" else x)

# Initialize a CamresV object using pandas.DataFrame
cramersv = am.CramersV(df) 

# return a pairwise matrix filled with Cramer's V, where columns and index are 
# the categorical variables of the passed pandas.DataFrame
cramersv.fit()

There is a strong correlation between the two features. I will choose to drop Status & Status2 feature and update Courier_Status to categorical data type.

In [ ]:
# Drop correlated features
sales_dummies.drop(['Status','Status2'], axis=1, inplace=True)

In [ ]:
# Verify features were dropped
sales_dummies.head(1)

In [ ]:
# Update to Courier_Status data type to category
sales_dummies['Courier_Status'] = sales_dummies['Courier_Status'].astype('category')

#### Correlation of Style and ASIN

In [ ]:
# Convert you str columns to Category columns
df1 = sales_dummies[{'Style','ASIN'}]
df1 = df1.apply(lambda x: x.astype("category") if x.dtype == "O" else x)

# Initialize a CamresV object
cramersv1 = am.CramersV(df1) 

# matrix filled with Cramer's V, where columns and index are 
# the categorical variables of the df
cramersv1.fit()

Highly correlated. I will drop ASIN.

#### Correlation of Style and Category

In [ ]:
# Convert you str columns to Category columns
df2 = sales_dummies[{'Style','Category'}]
df2 = df2.apply(
        lambda x: x.astype("category") if x.dtype == "O" else x)

# Initialize a CamresV object using you pandas.DataFrame
cramersv2 = am.CramersV(df2) 
# will return a pairwise matrix filled with Cramer's V, where columns and index are 
# the categorical variables of the passed pandas.DataFrame
cramersv2.fit()

Highly correlated. I will drop Style.

In [ ]:
# Drop correlated features
sales_dummies.drop(['ASIN'], axis=1, inplace=True)
sales_dummies.drop(['Style'], axis=1, inplace=True)

In [ ]:
# Verify dropped features
sales_dummies.head()

## Encoding

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Size' column
sales_dummies['Size_encoded'] = label_encoder.fit_transform(sales['Size'])

In [ ]:
# Initialize the LabelEncoder
label_encoder2 = LabelEncoder()

# Fit and transform the 'Size' column
sales_dummies['Category_encoded'] = label_encoder2.fit_transform(sales['Category'])

In [ ]:
# View the new encoded feature
sales_dummies.head()

In [ ]:
# Observe the Size value counts
sales_dummies['Size'].value_counts()

In [ ]:
# Observe the encoded value counts
sales_dummies['Size_encoded'].value_counts()

* 0, 3XL
* 1, 4XL
* 2, 5XL
* 3, 6XL
* 4, Free
* 5, L
* 6, M
* 7, S
* 8, XL
* 9, XS
* 10, XXL


In [ ]:
# Perform one-hot encoding for Courier_Status
encoded = pd.get_dummies(sales_dummies, columns=['Courier_Status'])

In [ ]:
# Drop the original features
encoded.drop(['Size', 'Category'], axis=1, inplace=True)

In [ ]:
# Sort the df in ascending date order
encoded = encoded.sort_values(by = ['Date'], ascending = True)

In [ ]:
# View the sorted df
encoded.head(5)

In [ ]:
encoded['Amount']=encoded['Amount'].round(2)

In [ ]:
encoded.head(3)

## Decomposition

In [ ]:
# Using decomposition to observe trends and seasonality in the data

result = seasonal_decompose(date_df['Amount'], period=7)
result.plot()

There is a slight downward trend.

## Dicky-Fuller test

Check to see if the data is stationary.

In [ ]:
adf, pval, usedlag, nobs, crit_vals, icbest =  adfuller(date_df['Amount'].values)
print('ADF test statistic:', adf)
print('ADF p-values:', pval)
print('ADF number of lags used:', usedlag)
print('ADF number of observations:', nobs)
print('ADF critical values:', crit_vals)
print('ADF best information criterion:', icbest)

The p-value is less than .05, therefore I'm determining it's stationary and differencing is not needed.


# Modeling

In [ ]:
date_df.head(1)

In [ ]:
pd.to_datetime(date_df['Date'])

In [ ]:
# Create train and test datasets
train_df = date_df[date_df['Date'] <= '2022-06-14']
test_df = date_df[date_df['Date'] > '2022-06-14']

In [ ]:
# View the training dataframe to observe dataframes were split correctly
pd.options.display.float_format = '{:.2f}'.format
train_df.tail()

In [ ]:
# View the test dataframe to observe dataframes were split correctly
test_df.head()

In [ ]:
# Adding date related features to the train dataset for modeling
train_df['dayofweek'] = train_df['Date'].dt.dayofweek
train_df['month'] = train_df['Date'].dt.month
train_df['dayofyear'] = train_df['Date'].dt.dayofyear
train_df['dayofmonth'] = train_df['Date'].dt.day
train_df['weekofyear'] = train_df['Date'].dt.weekofyear

# Adding date related features to the train dataset for modeling
test_df['dayofweek'] = test_df['Date'].dt.dayofweek
test_df['month'] = test_df['Date'].dt.month
test_df['dayofyear'] = test_df['Date'].dt.dayofyear
test_df['dayofmonth'] = test_df['Date'].dt.day
test_df['weekofyear'] = test_df['Date'].dt.weekofyear

In [ ]:
train_df.head()

## ARIMA

### Build the model

In [ ]:
# Create an ARIMA model with automated optimization of hyperparameters 
model = pm.auto_arima(train_df['Amount'], seasonal=False, stepwise=True)

### Predict Values

In [ ]:
# Predict retail sales based on the test dataset
preds = model.predict(test_df.shape[0])

In [ ]:
# View predicted values
preds

### Visualize the model

In [ ]:
forecast_df = pd.DataFrame(preds,index = test_df.index,columns=['preds'])

pd.concat([date_df['Amount'],forecast_df],axis=1).plot(color = ['skyblue','red'], 
                                                                title = 'Daily Amazon Sales with Auto ARIMA Predicted Sales')
# Name the x axis 
plt.xlabel('Index of Day') 
# Name the y axis 
plt.ylabel('Total Retail Sales per Day') 
plt.show()

### Evaluate the model

In [ ]:
# Calculate mean squared error of the test sales and predicted sales
mse = mean_squared_error(test_df['Amount'], preds)

# Calculate the square root of the mean quared error 
rmse = math.sqrt(mse)

# View root mean squared error
rmse

In [ ]:
# Create a function to calulate the weighted mean average percentage error
def wmape(y_true, y_pred):
        return np.sum(np.abs(y_true - y_pred))/ np.sum(np.abs(y_true))

In [ ]:
# Calculate the wmape as a second evaluation metric
wmape(test_df['Amount'], preds)

## Random Forest Regression

In [ ]:
# Attempt a Random Forest Regression model

# Set train variable
X = train_df[['dayofweek', 'month', 'dayofyear',
       'dayofmonth', 'weekofyear']]

# Fit the model
my_rf = RandomForestRegressor()
my_rf.fit(X, train_df['Amount'])

# Precit based on test data
X_test = test_df[['dayofweek', 'month', 'dayofyear',
       'dayofmonth', 'weekofyear']]
pred_test = my_rf.predict(X_test)

# View predicted values
pred_test

In [ ]:
# View actual values
test_df['Amount']

In [ ]:
# Calculate mse and rmse
rfr_mse = mean_squared_error(test_df['Amount'], pred_test)
rfr_rmse = math.sqrt(rfr_mse)

# View rmse
rfr_rmse

In [ ]:
# View wmape
wmape(test_df['Amount'], pred_test)

## Linear Regression

In [ ]:
# fit the Linear Regression model
my_lr = LinearRegression()
my_lr.fit(X, train_df['Amount'].values)

# Predict on the test period
predsLR = my_lr.predict(X_test)

# Calculate mse and rmse
LR_mse = mean_squared_error(test_df['Amount'], predsLR)
LR_rmse = math.sqrt(LR_mse)

# View rmse
LR_rmse

In [ ]:
# View wmape
wmape(test_df['Amount'], predsLR)

In [ ]:
# Forecast on the test data
test_df['Amount_Preds'] = predsLR

# Combine actual and predicted values in one dataframe
Amount_all2 = pd.concat([test_df, train_df], sort=False)

# Plot the acutal vs predicted values
_ = Amount_all2[['Amount','Amount_Preds']].plot(figsize=(15, 5))

## XGBoost

In [ ]:
# Set the date column as the index
date_index = date_df.set_index('Date')

In [ ]:
# Change date to datetime
pd.to_datetime(date_index.index)

In [ ]:
# Visualize sales over timeperiod
date_index['Amount'].plot(style='.', 
                          figsize=(15,5), 
                          title = "Amazon Sales per Day from March 31, 2022 to June 29, 2022")
plt.show()

In [ ]:
# Create a function to create time related features

def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
# Split the data into train and test sets based on 6/14/22 date
train = date_index.loc[date_index.index < '2022-06-14']
test = date_index.loc[date_index.index >= '2022-06-14']

In [ ]:
# Train, Test, Split
X_train, y_train = create_features(train, label='Amount')
X_test, y_test = create_features(test, label='Amount')

In [ ]:
# Fit the model
reg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], 
        verbose=False) # Change verbose to True if you want to see it train

In [ ]:
# Observe feature importance
_ = plot_importance(reg, height=0.9)

In [ ]:
# Forecast on the test data
test['Amount_Prediction'] = reg.predict(X_test)

In [ ]:
# Combine actual and predicted values in one dataframe
Amount_all = pd.concat([test, train], sort=False)

# Plot the acutal vs predicted values
_ = Amount_all[['Amount','Amount_Prediction']].plot(figsize=(15, 5))

In [ ]:
# Evaluate the model with mse
XGBmse = mean_squared_error(y_true=test['Amount'],
                   y_pred=test['Amount_Prediction'])

In [ ]:
# Calculate rmse
math.sqrt(XGBmse)

In [ ]:
# Evaluate the model with wmape
wmape(test['Amount'], test['Amount_Prediction'])